In [35]:
import numpy as np
from scipy.optimize import fsolve
from scipy.spatial.transform import Rotation 
from controller import PI_Controller
from scipy.integrate import odeint

In [36]:
def angle_wrap(angle):
    a = angle%(2*np.pi)
    if abs(a - 2*np.pi)<1e-3:
        a = 0
    return (a)

In [37]:
class PUMA():
    def __init__(self,d1=1,a2=1,a3=1):
        self.d1 = d1
        self.a2 = a2
        self.a3 = a3
        self.q1 = 0
        self.q2 = 0
        self.q3 = 0

        self.update_endEffectorPosition()

    def update_endEffectorPosition(self,):
        self.xE,self.yE,self.zE = self.forward_kinematics(self.q1,self.q2,self.q3)
    
    def inverse_kinematics(self,xc,yc,zc):

        # Workspace condition
        D = (xc**2 + yc**2 + (zc-self.d1)**2 - self.a2**2 - self.a3**2)/(2*self.a2*self.a3)
        if abs(D)<=1:
            def inv_func(x):
                return [
                        -xc + self.a2*np.cos(x[1])*np.cos(x[0]) + self.a3*np.cos(x[1]+x[2])*np.cos(x[0]),
                        -yc + self.a2*np.cos(x[1])*np.sin(x[0]) + self.a3*np.cos(x[1]+x[2])*np.sin(x[0]),
                        -zc + self.d1 + self.a2*np.sin(x[1]) + self.a3*np.sin(x[1]+x[2])
                        ]
            root = fsolve(inv_func,[0,0,0])
            q1,q2,q3 = root
            
            self.q1,self.q2,self.q3 = angle_wrap(q1),angle_wrap(q2),angle_wrap(q3)
            # Returns True if solution exists
            return True, [self.q1,self.q2,self.q3]
        else:
            # Returns True if solution exists
            print("Angles provided not in workspace")
            return False, [None, None, None]
    
    def forward_kinematics(self,q1,q2,q3):
        xc = self.a2*np.cos(q2)*np.cos(q1) + self.a3*np.cos(q2+q3)*np.cos(q1)
        yc = self.a2*np.cos(q2)*np.sin(q1) + self.a3*np.cos(q2+q3)*np.sin(q1)
        zc = self.d1 + self.a2*np.sin(q2) + self.a3*np.sin(q2+q3)

        self.xE,self.yE,self.zE = xc,yc,zc

        return xc,yc,zc
    
    def dynamics(self,):
        
        pass
    def point_tracking(self,trk_points):
        # dh = 
        # TMatrix = 
        xt,yt,zt = trk_points

        ret, angles = self.inverse_kinematics(xt,yt,zt)
        if ret:
            print(angles) 
        else:
            print('Error Occurred! Exiting ...')

In [69]:
class SCARA():
    def __init__(self,d=1,a1 = 1, a2=1):
        self.d = d
        self.a1 = a1
        self.a2 = a2

        self.m1 = 1
        self.m2 = 1
        self.m3 = 1

        self.q1 = 0
        self.q2 = 0

        self.q1_dot = 0
        self.q2_dot = 0
        self.d_dot = 0

        self.q1_d_dot = 0
        self.q2_d_dot = 0
        self.d_d_dot = 0

        self.tau_1 = 0
        self.tau_2 = 0
        self.F_3 = 0

        self.compute_inertias()

        self.update_endEffectorPosition()

        self.controller = PI_Controller([[1,0],
                                         [1,0],
                                         [1,0]])

    def compute_inertias(self,):
        self.J1 = (1/3)*self.m1*self.a1*2
        self.J2 = (1/3)*self.m2*self.a2*2
        self.J3 = (1/3)*self.m3*self.d*2

    def update_endEffectorPosition(self,):
        self.xE,self.yE,self.zE = self.forward_kinematics(self.q1,self.q2,self.d)

    def inverse_kinematics(self,xc,yc,zc):
        # Workspace condition
        if np.sqrt(xc**2+yc**2)>self.a2 + self.a1:
            print("No Solution can be Found!")
            return False,[None, None, None]
        else: 
            def inv_func(x):
                return [
                        - xc + self.a1*np.cos(x[0]) + self.a2*np.cos(x[0]+x[1]),
                        - yc + self.a1*np.sin(x[0]) + self.a2*np.sin(x[0]+x[1]),
                        - zc + self.d - x[2]
                        ]
            root = fsolve(inv_func,[1,1,1])

            q1,q2,d = root

            return True, [angle_wrap(q1),angle_wrap(q2),d]
    
    def forward_kinematics(self,q1,q2,d):

        xc = self.a1*np.cos(q1) + self.a2*np.cos(q1+q2)
        yc = self.a1*np.sin(q1) + self.a2*np.sin(q1+q2)
        zc = self.d - d

        self.xE,self.yE,self.zE = xc,yc,zc

        return xc,yc,zc
    
    def dynamics_solver(self,inp,t,d_q1,d_q2,d_d3):

        g  = 9.8
        q1,q1_dot = inp[:2]
        q2,q2_dot = inp[2:4]
        d3,d3_dot = inp[4:]
        
        q1 = angle_wrap(q1)
        q2 = angle_wrap(q2)

        alpha = self.J1 + self.a1**2*(self.m1/4 + self.m2 + self.m3)
        beta = self.J2 + self.J3 + self.a2**2*(self.m2/4 +self.m3) 
        gamma = self.a1*self.a2*self.m3 + self.a1 * self.a2/2 * self.m2

        e = [d_q1 - self.q1,
             d_q2 - self.q2,
             d_d3 - self.d]

        self.tau_1,self.tau_2,self.F_3 = self.controller.track_angles(e)

        MM = np.array([[alpha + beta + 2*gamma*np.cos(q2), beta + 2*gamma*np.cos(q2), 0],
                       [beta + 2*gamma*np.cos(q2), beta, 0],
                       [0, 0, self.m3]])
                       
        C = np.array([[-gamma*np.sin(q2)*q2_dot, -gamma*np.sin(q2)*(q2_dot + q1_dot), 0],
                      [gamma*np.sin(q2)*q1_dot, 0, 0],
                      [0, 0, 0]])
        G = np.transpose(np.array([0, 0, self.m3*g]))

        U = np.transpose(np.array([self.tau_1, self.tau_2, self.F_3]))

        K = (U - np.matmul(C, np.transpose([q1_dot, q2_dot, d3_dot])) - G)

        d2ydt2 = np.matmul(np.linalg.inv(MM), K)

        yd = np.array([q1_dot, d2ydt2[0], 
                       q2_dot, d2ydt2[1], 
                       d3_dot, d2ydt2[2]])

        return yd


    
    def point_tracking(self,trk_points):

        xt,yt,zt = trk_points

        ret, angles = self.inverse_kinematics(xt,yt,zt)

        init_cond = np.array([self.q1,self.q1_dot,
                              self.q2,self.q2_dot,
                              self.d ,self.d_dot ])
                              
        t = np.linspace(0,50,num=1000 )

        y = odeint(self.dynamics_solver,init_cond,t, args=(angles[0],angles[1],angles[2]))
        print(y)

In [71]:
a = SCARA()
a.point_tracking([0.19,0,0.1])

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [ 5.51637365e-04  2.20434290e-02  2.38663023e-04  9.53697333e-03
   9.87600213e-01 -4.95495495e-01]
 [ 2.20654843e-03  4.40868081e-02  9.54653857e-04  1.90740370e-02
   9.50400851e-01 -9.90990991e-01]
 ...
 [ 4.39439439e+01  4.39939940e+01  4.40440440e+01  4.40940941e+01
   4.41441441e+01  4.41941942e+01]
 [ 4.42442442e+01  4.42942943e+01  4.43443443e+01  4.43943944e+01
   4.44444444e+01  4.44944945e+01]
 [ 4.45445445e+01  4.45945946e+01  4.46446446e+01  4.46946947e+01
   4.47447447e+01  4.47947948e+01]]


C:\Users\rwik2\anaconda3\envs\Rwik1234\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
